# POS-tagger и NER

## Задание 1. Написать теггер на данных с русским языком

In [1]:
import pandas as pd

import pyconll
from corus import load_ne5

import nltk
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
from nltk.tag import SequentialBackoffTagger

import spacy
from spacy import displacy
import ru_core_news_sm

import deeppavlov
from deeppavlov import configs, build_model

from razdel import tokenize

nltk.download('words')
nltk.download('maxent_ne_chunker')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\avpat\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\avpat\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\avpat\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [3]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации

In [4]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.accuracy(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.accuracy(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.accuracy(fdata_test)

bigram_unigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_unigram_tagger.accuracy(fdata_test)

trigram_bigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_acc = trigram_bigram_tagger.accuracy(fdata_test)

In [5]:
print('Accuracy:')
print(f'\tUnigramTagger: {unigram_acc:.4f}')
print(f'\tBigramTagger: {bigram_acc:.4f}')
print(f'\tTrigramTagger: {trigram_acc:.4f}')
print(f'\tBigramTagger + UnigramTagger: {bigram_unigram_acc:.4f}')
print(f'\tTrigramTagger + BigramTagger: {trigram_bigram_acc:.4f}')

Accuracy:
	UnigramTagger: 0.8237
	BigramTagger: 0.6094
	TrigramTagger: 0.1779
	BigramTagger + UnigramTagger: 0.8293
	TrigramTagger + BigramTagger: 0.6092


2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

In [6]:
class MyTagger(SequentialBackoffTagger):
    def __init__(self, dataset, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)

        self.tag_dict = {}
        for line in dataset:
            for tag in line:
                if not tag[1] in self.tag_dict.keys():
                    self.tag_dict[tag[1]] = set()
                
                if tag[1] and type(tag[0]) is str and type(tag[1] is str):
                    self.tag_dict[tag[1].upper()].add(tag[0].lower())
        
        
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if type(word) is str:
            word.lower()
            for tag in self.tag_dict.keys():
                if word in self.tag_dict[tag]:
                    return tag
        
        return None

tagger = MyTagger(fdata_train)
my_tagger_acc = tagger.accuracy(fdata_test)
print(f'MyTagger: {my_tagger_acc:.4f}')

MyTagger: 0.6498


3. сравнить все реализованные методы, сделать выводы

In [7]:
taggers = {
    'Tagger': ['UnigramTagger', 'BigramTagger', 'TrigramTagger',
               'BigramTagger + UnigramTagger', 'TrigramTagger + BigramTagger', 'MyTagger'],
    'Accuracy': [unigram_acc, bigram_acc, trigram_acc, bigram_unigram_acc, trigram_bigram_acc, my_tagger_acc]
}
results = pd.DataFrame(taggers)
results.sort_values('Accuracy', ascending=False)

,Tagger,Accuracy
3,BigramTagger + UnigramTagger,0.829279
0,UnigramTagger,0.823732
5,MyTagger,0.649840
1,BigramTagger,0.609389
4,TrigramTagger + BigramTagger,0.609180
2,TrigramTagger,0.177863


Наилучшим образом себя показали Unigram теггер и его комбинация с Bigram тегерром. Остальные показали себя на этом датасете значительно хуже.

## Задание 2. Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/

In [8]:
# Пришлось влезть в corus и поставить кодировку т.к. падало с ошибкой
data = load_ne5('./datasets/Collection5/')
document = next(data).text
print(document)

Россия рассчитывает на конструктивное воздействие США на Грузию

04/08/2008 12:08

МОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.

"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.

"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. 


1. проверить NER из nltk/spacy/deeppavlov.

<b>nltk</b>

In [9]:
res = set()
pos_tag = nltk.pos_tag(nltk.word_tokenize(document))
for chunk in nltk.ne_chunk(pos_tag):
    if hasattr(chunk, 'label'):
        text = ' '.join([c[0] for c in chunk])
        label = chunk.label()
        res.add((text, label))

res

{('МИД России', 'ORGANIZATION'),
 ('МОСКВА', 'ORGANIZATION'),
 ('РИА Новости', 'ORGANIZATION'),
 ('России Григорий Карасин', 'PERSON'),
 ('Россия', 'PERSON'),
 ('Тбилиси', 'PERSON')}

<b>spacy</b>

In [10]:
sp = ru_core_news_sm.load()
displacy.render(sp(document), jupyter=True, style='ent')

<b>deeppavlov</b>

In [11]:
# !python -m venv env 
# # !.\env\Scripts\activate.bat
# !pip install deeppavlov
# !python -m deeppavlov install squad_bert

# !python -m deeppavlov install ner_ontonotes
cfg = configs.ner.ner_rus_convers_distilrubert_6L
deeppavlov_ner = build_model(cfg, download=True)
res = deeppavlov_ner([document])

2023-05-20 21:42:53.108 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_rus_conversational_distilrubert_6L.tar.gz download because of matching hashes
Some weights of the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificati

In [12]:
res = {
    'words': res[0][0],
    'labels': res[1][0]
}
res = pd.DataFrame(res)
res[res['labels'] != 'O']

,words,labels
0,Россия,B-LOC
5,США,B-LOC
7,Грузию,B-LOC
24,МОСКВА,B-LOC
29,РИА,B-ORG
30,Новости,I-ORG
32,Россия,B-LOC
36,США,B-LOC
39,Тбилиси,B-LOC
62,России,B-LOC
